In [1]:
import numpy as np
import pandas as pd
import warnings
import pycountry
warnings.filterwarnings(action="ignore")
pd.options.display.max_colwidth = 200
pd.set_option('display.max_columns', 30)
%load_ext autotime

In [2]:
def get_country_from_ISO(iso_code):
    try:
        return pycountry.countries.get(alpha_2=iso_code).name
    except:
        return ""

time: 1.79 ms


In [3]:
output_df = pd.read_csv("120719 - 92 Keys/120719 - 92 Keys - Google Ads - {} Day.csv".format(30), skiprows=2)[['Campaign', 'Ad group', "Ad group max. CPV"]]
output_df["Country"] = output_df['Campaign'].str.split("|", expand=True)[2].str.strip()
output_df['Ad group max. CPV'] *= 0.055

time: 296 ms


In [4]:
for day in [1, 7, 14, 30]:
    google = pd.read_csv("120719 - 92 Keys/120719 - 92 Keys - Google Ads - {} Day.csv".format(day), 
                           skiprows=2)[['Campaign', 'Ad group', "YouTube Earned Views", "Views"]]
    output_df = output_df.merge(google, on=["Campaign", "Ad group"]).rename(columns={"YouTube Earned Views": 
                                                                                       "YouTube Earned Views {} days".format(day), 
                                                                                     'Views': "Ad Views {} days".format(day)})

time: 535 ms


In [5]:
for day in [1, 7, 14, 30]:
    youtube = pd.read_csv("120719 - 92 Keys/120719 - 92 Keys - YouTube - {} Day.csv".format(day))
    google = pd.read_csv("120719 - 92 Keys/120719 - 92 Keys - Google Ads - {} Day.csv".format(day), skiprows=2)
    google["Country"] = google['Campaign'].str.split("|", expand=True)[2].str.strip()
    google['Cost'] *= 0.055
    google['Ad group max. CPV'] *= 0.055
    google['Avg. CPV'] *= 0.055
    google_ads_country_groups = google.groupby("Country").sum().reset_index()
    youtube['Country'] = youtube['Geography'].apply(get_country_from_ISO)
    combined_df_country_groups = google_ads_country_groups.merge(youtube, on="Country", how="left", suffixes=("_Google", "_Youtube"))
    combined_df_country_groups["Country ROAS {} days".format(day)] = combined_df_country_groups['Your estimated revenue (USD)']/combined_df_country_groups['Cost']
    df = combined_df_country_groups[['Country', 'Cost', 'Views_Google', 'YouTube Earned Views', 'Your estimated revenue (USD)', "Country ROAS {} days".format(day)]]
    df["Earned View Value {} days".format(day)] = df['Your estimated revenue (USD)']/df['YouTube Earned Views']
    df["Earned View Cost {} days".format(day)] = df['Cost']/df['YouTube Earned Views']
    output_df = output_df.merge(df[["Country", "Earned View Value {} days".format(day), "Earned View Cost {} days".format(day), 
                                    "Country ROAS {} days".format(day)]], on="Country", how="left")

time: 1.21 s


In [6]:
def calculate_max_bid(row):
    weights = {}
    minimum_number_views = 10
    type_of_view = "Ad Views"
    for day in [1, 7, 14, 30]:
        weights[day] = 0
    if row[type_of_view + ' 1 days'] >= minimum_number_views:
        weights[1] = .4
    if row[type_of_view + ' 7 days'] >= minimum_number_views:
        weights[7] = .3
    if row[type_of_view + ' 14 days'] >= minimum_number_views:
        weights[14] = .2
    if row[type_of_view + ' 30 days'] >= minimum_number_views:
        weights[30] = .1
    total_weight = np.sum(list(weights.values()))
    if total_weight == 0:
        return row['Ad group max. CPV']
#     new_max_bid = 0
    country_ROAS = 0
    earned_view_value_over_cost = 0
    for day in [1, 7, 14, 30]:
        weights[day] /= total_weight
        country_ROAS += row["Country ROAS {} days".format(day)] * weights[day]
        earned_view_value_over_cost += ((row["Earned View Value {} days".format(day)]) / (row["Earned View Cost {} days".format(day)])) * weights[day]
        
    if (earned_view_value_over_cost >= 1) or (country_ROAS >= 2):
        return row['Ad group max. CPV'] * 1.1
    elif earned_view_value_over_cost <= .5:
        return row['Ad group max. CPV'] * 0.9
    else:
        return row['Ad group max. CPV']
    
# #         new_max_bid += row["Earned View Value {} days".format(day)] * weights[day]
        
# #         if np.isnan(new_max_bid) or np.isinf(new_max_bid):
# #             return row['Ad group max. CPV']
    
#     return new_max_bid

time: 4.53 ms


In [7]:
output_df["Max Bid"] = output_df.apply(calculate_max_bid, axis=1)

time: 3.02 s


In [9]:
new_max_bid_df = output_df[output_df['Max Bid'] != output_df['Ad group max. CPV']][['Campaign', 'Ad group', 'Max Bid']]

time: 17.5 ms


In [32]:
new_max_bid_df.to_csv("new_max_bid_ads.csv", index=False)

time: 16.8 ms


In [124]:
dfs[3]

,Country,Cost,Views_Google,YouTube Earned Views,Your estimated revenue (USD),ROAS_30,Earned View Value 30 days
0,Australia,1.23640,199,16,2.094,1.693627,0.130875
1,Austria,0.83325,318,30,2.455,2.946295,0.081833
2,Belgium,0.02310,9,2,1.204,52.121212,0.602000
3,Canada,1.90245,264,31,7.915,4.160425,0.255323
4,Czechia,1.24190,369,42,2.064,1.661970,0.049143
5,Denmark,0.09625,26,4,0.997,10.358442,0.249250
6,Finland,0.01980,6,1,0.246,12.424242,0.246000
7,France,1.05765,299,33,2.999,2.835532,0.090879
8,Germany,1.35520,264,36,4.314,3.183294,0.119833
9,Hong Kong,0.35090,116,21,1.135,3.234540,0.054048


time: 18.9 ms


In [99]:
youtube_30_day = pd.read_csv("120719 - 92 Keys/120719 - 92 Keys - YouTube - 30 Day.csv")
google_30_day = pd.read_csv("120719 - 92 Keys/120719 - 92 Keys - Google Ads - 30 Day.csv", skiprows=2)

time: 159 ms


In [100]:
google_30_day["Country"] = google_30_day['Campaign'].str.split("|", expand=True)[2].str.strip()

time: 228 ms


In [101]:
google_30_day['Cost'] *= 0.055

time: 4.78 ms


In [102]:
google_ads_country_groups = google_30_day.groupby("Country").sum().reset_index()

time: 35.8 ms


In [104]:
combined_df_country_groups = google_ads_country_groups.merge(youtube_30_day, on="Country", how="left", suffixes=("_Google", "_Youtube"))

time: 11.5 ms


In [113]:
combined_df_country_groups['ROAS'] = combined_df_country_groups['Your estimated revenue (USD)']/combined_df_country_groups['Cost']

time: 1.56 ms


In [114]:
df = combined_df_country_groups[['Country', 'Cost', 'Views_Google', 'YouTube Earned Views', 'Your estimated revenue (USD)', 'ROAS']]

time: 4.18 ms


In [115]:
df['Earned View Value 30 days'] = df['Your estimated revenue (USD)']/df['YouTube Earned Views']

time: 3.62 ms


In [116]:
df

,Country,Cost,Views_Google,YouTube Earned Views,Your estimated revenue (USD),ROAS,Earned View Value 30 days
0,Australia,1.23640,199,16,2.094,1.693627,0.130875
1,Austria,0.83325,318,30,2.455,2.946295,0.081833
2,Belgium,0.02310,9,2,1.204,52.121212,0.602000
3,Canada,1.90245,264,31,7.915,4.160425,0.255323
4,Czechia,1.24190,369,42,2.064,1.661970,0.049143
5,Denmark,0.09625,26,4,0.997,10.358442,0.249250
6,Finland,0.01980,6,1,0.246,12.424242,0.246000
7,France,1.05765,299,33,2.999,2.835532,0.090879
8,Germany,1.35520,264,36,4.314,3.183294,0.119833
9,Hong Kong,0.35090,116,21,1.135,3.234540,0.054048


time: 17.4 ms


In [88]:
google_30_day.merge(df, on="Country", how="left")

,Campaign,Ad group,Ad group status,Ad group max. CPV,Currency,Impressions,View rate,Cost_x,Avg. CPV,Views,YouTube Earned Views_x,Country,Cost_y,Views_Google,YouTube Earned Views_y,Your estimated revenue (USD),ROAS,Earned View Value 30 days
0,92 Keys | YouTube | Portugal | Videos,Christmas Bumper | Topics | Religious Music | ...,Campaign paused,0.04,MXN,0,0.00%,0.0,0.0,0,0,Portugal,0.3289,215,38,0.937,0.351014,0.024658
1,92 Keys | YouTube | Portugal | Videos,Christmas Bumper | Affinity | Classical Fans |...,Campaign paused,0.04,MXN,0,0.00%,0.0,0.0,0,0,Portugal,0.3289,215,38,0.937,0.351014,0.024658
2,92 Keys | YouTube | Portugal | Videos,Christmas Bumper | Affinity | Classical Fans |...,Campaign paused,0.04,MXN,0,0.00%,0.0,0.0,0,0,Portugal,0.3289,215,38,0.937,0.351014,0.024658
3,92 Keys | YouTube | Portugal | Videos,Christmas Bumper | Affinity | Classical Fans |...,Campaign paused,0.04,MXN,0,0.00%,0.0,0.0,0,0,Portugal,0.3289,215,38,0.937,0.351014,0.024658
4,92 Keys | YouTube | Portugal | Videos,Christmas Bumper | Affinity | Classical Fans |...,Campaign paused,0.04,MXN,0,0.00%,0.0,0.0,0,0,Portugal,0.3289,215,38,0.937,0.351014,0.024658
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40819,92 Keys | YouTube | Belgium | Search,Christmas Bumper | Topics | Classical Music | ...,Campaign paused,0.08,MXN,0,0.00%,0.0,0.0,0,0,Belgium,0.0231,9,2,1.204,0.019186,0.602000
40820,92 Keys | YouTube | Belgium | Search,Christmas Bumper | Topics | Classical Music | ...,Campaign paused,0.08,MXN,0,0.00%,0.0,0.0,0,0,Belgium,0.0231,9,2,1.204,0.019186,0.602000
40821,92 Keys | YouTube | Belgium | Search,Christmas Bumper | Topics | Classical Music | ...,Campaign paused,0.08,MXN,0,0.00%,0.0,0.0,0,0,Belgium,0.0231,9,2,1.204,0.019186,0.602000
40822,92 Keys | YouTube | Belgium | Search,Christmas Bumper | Topics | Classical Music | ...,Campaign paused,0.08,MXN,0,0.00%,0.0,0.0,0,0,Belgium,0.0231,9,2,1.204,0.019186,0.602000


time: 92.7 ms


In [71]:
df

,Country,Cost,Views_Google,YouTube Earned Views,Your estimated revenue (USD),ROAS,Earned View Value
0,Australia,1.23640,199,16,2.094,0.590449,0.130875
1,Austria,0.83325,318,30,2.455,0.339409,0.081833
2,Belgium,0.02310,9,2,1.204,0.019186,0.602000
3,Canada,1.90245,264,31,7.915,0.240360,0.255323
4,Czechia,1.24190,369,42,2.064,0.601696,0.049143
5,Denmark,0.09625,26,4,0.997,0.096540,0.249250
6,Finland,0.01980,6,1,0.246,0.080488,0.246000
7,France,1.05765,299,33,2.999,0.352668,0.090879
8,Germany,1.35520,264,36,4.314,0.314140,0.119833
9,Hong Kong,0.35090,116,21,1.135,0.309163,0.054048


time: 29.2 ms


In [ ]:
earned_view_by_country_1_day = google_1_day.groupby("Country").sum()[['YouTube Earned Views']].reset_index().sort_values("YouTube Earned Views", ascending=False).reset_index(drop=True)